### Importing the Packages

In [32]:
import pandas as pd
import numpy as np

### Importing the data 

In [ ]:
raw_csv_data = pd.read_csv("Index2018.csv")
df_comp = raw_csv_data.copy()

### Length of the Time Period

In [ ]:
df_comp["date"].describe()

count           6269
unique          6269
top       29/01/2018
freq               1
Name: date, dtype: object

### From Text to Date

In [ ]:
df_comp["date"] = pd.to_datetime(df_comp["date"], dayfirst=True)

In [36]:
df_comp.head()

,date,spx,dax,ftse,nikkei
0,1994-01-07,469.90,2224.95,3445.98,18124.01
1,1994-01-10,475.27,2225.00,3440.58,18443.44
2,1994-01-11,474.13,2228.10,3413.77,18485.25
3,1994-01-12,474.17,2182.06,3372.02,18793.88
4,1994-01-13,472.47,2142.37,3360.01,18577.26


In [37]:
df_comp["date"].describe()

count                             6269
mean     2006-01-14 19:36:59.492742144
min                1994-01-07 00:00:00
25%                2000-01-11 00:00:00
50%                2006-01-12 00:00:00
75%                2012-01-19 00:00:00
max                2018-01-29 00:00:00
Name: date, dtype: object

In [ ]:
df_comp["date"].nunique()

6269

### Setting the Index

In [ ]:
df_indexed = df_comp.set_index("date")
df_indexed.head()

,spx,dax,ftse,nikkei
date,,,,
1994-01-07,469.90,2224.95,3445.98,18124.01
1994-01-10,475.27,2225.00,3440.58,18443.44
1994-01-11,474.13,2228.10,3413.77,18485.25
1994-01-12,474.17,2182.06,3372.02,18793.88
1994-01-13,472.47,2142.37,3360.01,18577.26


### Setting the Desired Frequency

In [40]:
# Set frequency to business days. Trading does not occur on weekends and will lead to NaN values.
df_indexed = df_indexed.asfreq("b")
df_indexed.head()

,spx,dax,ftse,nikkei
date,,,,
1994-01-07,469.90,2224.95,3445.98,18124.01
1994-01-10,475.27,2225.00,3440.58,18443.44
1994-01-11,474.13,2228.10,3413.77,18485.25
1994-01-12,474.17,2182.06,3372.02,18793.88
1994-01-13,472.47,2142.37,3360.01,18577.26


### Handling Missing Values
- This is for illustration only. Practically, a consistent approach should be used.

In [41]:
df_filled = df_indexed.copy()
df_filled.isna().sum()

spx       8
dax       8
ftse      8
nikkei    8
dtype: int64

In [ ]:
# Forward fill spx
df_filled["spx"] = df_filled["spx"].ffill()
df_filled.isna().sum()

spx       0
dax       8
ftse      8
nikkei    8
dtype: int64

In [43]:
# Backfill ftse
df_filled["ftse"] = df_filled["ftse"].bfill()
df_filled.isna().sum()

spx       0
dax       8
ftse      0
nikkei    8
dtype: int64

In [ ]:
# Fill dax and nikkei with their mean values
for ticker in ["dax", "nikkei"]:
    mean_price = df_filled[ticker].mean()
    df_filled[ticker] = df_filled[ticker].fillna(mean_price)

df_filled.isna().sum()

spx       0
dax       0
ftse      0
nikkei    0
dtype: int64

### Simplifying the Dataset

Simplify the dataframe to keep only `spx` market values.

In [ ]:
df_spx = df_indexed[["spx"]].rename({"spx": "market_value"}, axis="columns")
df_spx.head()

,market_value
date,
1994-01-07,469.90
1994-01-10,475.27
1994-01-11,474.13
1994-01-12,474.17
1994-01-13,472.47


In [46]:
df_spx.describe()

,market_value
count,6269.000000
mean,1288.127542
std,487.586473
min,438.920000
25%,990.671905
50%,1233.420000
75%,1459.987747
max,2872.867839


### Splitting the Data

- Time series data cannot be shuffled.
- A cut off point is used. Data prior to this is assigned to the training set. Data following this is assigned to the testing set.
- An 80:20 split for training and testing data is reasonable to prevent overfitting and maintain accuracy.

In [52]:
train_size = int(df_spx.shape[0] * 0.8)
train_size

5021

In [57]:
df_train = df_spx.iloc[:train_size]
df_test = df_spx.iloc[train_size:]

Compare the tail of df_train with the head of df_test to ensure the last value does not overlap

In [58]:
df_train.tail()

,market_value
date,
2013-04-01,1562.173837
2013-04-02,1570.252238
2013-04-03,1553.686978
2013-04-04,1559.979316
2013-04-05,1553.278930


In [59]:
df_test.head()

,market_value
date,
2013-04-08,1563.071269
2013-04-09,1568.607909
2013-04-10,1587.731827
2013-04-11,1593.369863
2013-04-12,1588.854623
